In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import numpy as np

### Prediction for all datasets except SQ-WD given known entity and relation

In [13]:
import requests

def run_query_object(entity_idx, property_idx):
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT ?object  WHERE {

   wd:<ENTITY> wdt:<PROPERTY> ?object .} 
        """.replace(
            "<ENTITY>", entity_idx).replace(
            "<PROPERTY>", property_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    objects = []
    for q in range(len(data["results"]['bindings'])):
        objects.append(data["results"]['bindings'][q]['object']['value'])                 
    return ', '.join(objects)

In [14]:
data = pd.read_csv('rubq_paper_res_sel_lbl_gen.csv')
data['pred_gen_mgenre'] = np.nan

In [15]:
for k in tqdm(range(len(data))):
    if pd.isna(data.loc[k, 'pred_gen_mgenre']):
        try:
            cur_subjects = data.loc[k, 'pred_on_wdsq_and_rubq_for_rubq_ner_upd'].split(', ')
            for w in range(len(cur_subjects)):
                time.sleep(1)
                cur_subj = data.loc[k, 'pred_on_wdsq_and_rubq_for_rubq_ner_upd'].split(', ')[w]
                cur_rel = data.loc[k, 'prop_gen'].split(', ')[0]
                obj_pred = run_query_object(cur_subj, cur_rel)
                if len(obj_pred) > 0:
                    data.loc[k, 'pred_gen_mgenre'] = obj_pred
                    break
        except:
            time.sleep(1)
            continue

100%|██████████| 1186/1186 [49:33<00:00,  2.51s/it] 


In [18]:
for j in tqdm(range(len(data))):
    if pd.notna(data.loc[j, 'pred_gen_mgenre']):
        cands = data.loc[j, 'pred_gen_mgenre'].split(', ')
        elems = []
        for k in range(len(cands)):
            if 'entity/' in cands[k]:
                elems.append(cands[k].split('entity/')[1])
            else:
                elems.append(cands[k])
        data.loc[j, 'pred_gen_mgenre_lbl'] = ', '.join(elems)

100%|██████████| 1186/1186 [00:00<00:00, 7425.13it/s]


In [20]:
count = 0
for i in range(len(data)):
    if data.loc[i, 'answers'] == data.loc[i, 'pred_gen_mgenre_lbl']:
        count += 1
print(count/len(data))

0.47554806070826305


In [21]:
data.to_csv('rubq_paper_res_sel_lbl_gen.csv, index = False')

### SQ-WD

In [30]:
wdsq = pd.read_csv('wdsq_paper_res_sel_lbl_gen.csv')

In [ ]:
import requests

def run_query_object(entity_idx, property_idx):
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT ?object  WHERE {

   wd:<ENTITY> wdt:<PROPERTY> ?object .} 
        """.replace(
            "<ENTITY>", entity_idx).replace(
            "<PROPERTY>", property_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    objects = []
    for q in range(len(data["results"]['bindings'])):
        objects.append(data["results"]['bindings'][q]['object']['value'])                 
    return ', '.join(objects)

def run_query_subject(entity_idx, property_idx):
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT ?object  WHERE {

   ?object wdt:<PROPERTY> wd:<ENTITY> .} 
        """.replace(
            "<ENTITY>", entity_idx).replace(
            "<PROPERTY>", property_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    objects = []
    for q in range(len(data["results"]['bindings'])):
        objects.append(data["results"]['bindings'][q]['object']['value'])                 
    return ', '.join(objects)

In [35]:
wdsq['pred_obj_gen_mgenre'] = np.nan

In [36]:
for k in tqdm(range(len(wdsq))):
    if pd.isna(wdsq.loc[k, 'pred_obj_gen_mgenre']):
        try:
            cur_subjects = wdsq.loc[k, 'pred_on_wdsq_for_wdsq_ner'].split(', ')
            for w in range(len(cur_subjects)):
                time.sleep(1)
                cur_subj = cur_subjects[w]
                cur_rel = wdsq.loc[k, 'prop_gen'].split(', ')[0]
                if cur_rel[0] == "P":
                    obj_pred = run_query_object(cur_subj, cur_rel)
                    if len(obj_pred) > 0:
                        wdsq.loc[k, 'pred_obj_gen_mgenre'] = obj_pred
                        break
                else:
                    cur_rel = "P"+cur_rel[1:]
                    subj_pred = run_query_subject(cur_subj, cur_rel)
                    if len(subj_pred) > 0:
                        wdsq.loc[k, 'pred_obj_gen_mgenre'] = subj_pred
                        break                  
        except:
            time.sleep(1)
            continue

100%|██████████| 2491/2491 [1:21:07<00:00,  1.95s/it]


In [39]:
for j in tqdm(range(len(wdsq))):
    if pd.notna(wdsq.loc[j, 'pred_obj_gen_mgenre']):
        cands = wdsq.loc[j, 'pred_obj_gen_mgenre'].split(', ')
        elems = []
        for k in range(len(cands)):
            if 'entity/' in cands[k]:
                elems.append(cands[k].split('entity/')[1])
            else:
                elems.append(cands[k])
        wdsq.loc[j, 'pred_obj_gen_mgenre_lbl'] = ', '.join(elems)

100%|██████████| 2491/2491 [00:00<00:00, 8455.42it/s]


In [41]:
count = 0
for i in range(len(wdsq)):
    if pd.notna(wdsq.loc[i, 'pred_obj_gen_mgenre_lbl']):
        if wdsq.loc[i, 'object'] == wdsq.loc[i, 'pred_obj_gen_mgenre_lbl'].split(', ')[0]:
            count += 1
print(count/len(wdsq))

0.5748695303091128


In [42]:
wdsq.to_csv('wdsq_paper_res_sel_lbl_gen.csv', index = False)